In [16]:
import os
import pandas as pd
from dask import delayed
from dask import compute

In [30]:
from dask.distributed import Client

client = Client(n_workers=4)

/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
sorted(os.listdir(os.path.join('example-data', 'iris')))

['iris_1.csv', 'iris_2.csv', 'iris_3.csv']

In [6]:
from glob import glob
filenames = sorted(glob(os.path.join('example-data', 'iris', '*.csv')))

In [7]:
filenames

['example-data/iris/iris_1.csv',
 'example-data/iris/iris_2.csv',
 'example-data/iris/iris_3.csv']

In [10]:
df = pd.read_csv(os.path.join('example-data', 'iris', 'iris_1.csv'))
df.head()

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


# standard pandas group-by within a forloop

In [11]:
%%time

sums = []
avg = []
for fn in filenames:
    # Read in file
    df = pd.read_csv(fn)

    # Groupby origin airport
    by_variety = df.groupby('variety')

    # Sum of all departure delays by origin
    total = by_variety.sepal_length.sum()

    # Number of flights by origin
    mean = by_variety.sepal_length.mean()

    # Save the intermediates
    sums.append(total)
    avg.append(mean)

# Combine intermediates
total_sepal_length = sum(sums)
average_sepal_length = sum(avg)

result = average_sepal_length/total_sepal_length

CPU times: user 21.1 ms, sys: 9.53 ms, total: 30.6 ms
Wall time: 52.2 ms


# dask approach
Note. Calling the .compute() method works well when you have a single output. 

When you have multiple outputs you might want to use the dask.compute function:

In [31]:
#%%time

# This is just one possible solution, there are
# several ways to do this using `delayed`

sums = []
avg = []
for fn in filenames:
    # Read in file
    df = delayed(pd.read_csv)(fn)

    # Groupby origin airport
    by_variety = df.groupby('variety')

    # Sum of all departure delays by origin
    total = by_variety.sepal_length.sum()

    # Number of flights by origin
    mean = by_variety.sepal_length.mean()

    # Save the intermediates
    sums.append(total)
    avg.append(mean)

# Compute the intermediates
sums, avg = compute(sums, avg)

# Combine intermediates 
total_sepal_length = sum(sums)
average_sepal_length = sum(avg)

result = average_sepal_length/total_sepal_length

In [28]:
#mean.visualize()
#total.visualize()

In [32]:
client.close()